# Using ROOT to bind Python and C++

PyROOT is not just a set of Python bindings to C++ functions; it dynamically generates bindings for all C++ classes and functions in its namespace.

## What is PyROOT?

* Python bindings offered by ROOT
* Access all the (not only!) ROOT C++ functionality from Python (Python façade, C++ performance)
* Automatic, dynamic 
 * No static wrapper generation
 * Dynamic python proxies for C++ entities
 * Lazy class/variable lookup

* Powered by the ROOT type system and Cling
 * Reflection information, JIT C++ compilation, execution
* Pythonizations
 * Make it simpler, more pythonic

<center><img src="images/pyroot.png"></center>

Let's start with a part learning how to include new functions that you define.

In [ ]:
import ROOT

In a script, use `gInterpreter.ProcessLine` or `gInterpreter.Declare` to run C++ code.

In [ ]:
ROOT.gInterpreter.Declare("""
double dostuff(double a, double b) {
    return a + b;
}
""")

In [ ]:
ROOT.dostuff(3.14, 100)

Suppose we want to change the `dostuff` function, common on a prompt or in a notebook.

In [ ]:
ROOT.gInterpreter.Declare("""
double dostuff(double a, double b) {
    return (a + b) / sqrt(a*a + b*b);
}
""")

Ouch! Of course it doesn't let us replace a definition (not legal in C++), but we really want to.

This is what I do:

In [ ]:
tmpname = "dostuffv1"                  # keep changing the C++ name

ROOT.gInterpreter.Declare("""
double %s(double a, double b) {
    return (a + b) / sqrt(a*a + b*b);
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)       # but assign to the same Python name

In [ ]:
dostuff(3.14, 100)

<center><img src="images/03-cheat-sheet.png"></center>

## ROOT and Numpy arrays

Passing between C++ and Python is an expensive process (looking up class and function definitions in ROOT's C++ reflection and dynamically generating equivalent Python structures).

To use these bindings efficiently, we should send bulk data over the boundary and do loops on the C++ side.

Python's bulk container is the Numpy array.

In [ ]:
import numpy
a = numpy.arange(10) * 1.1
a

In [ ]:
tmpname = "dostuffv2"

ROOT.gInterpreter.Declare("""
void %s(int num, double *data) {
    for (int i = 0;  i < num;  i++) {
        data[i] += 100;
    }
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)

In [ ]:
dostuff(len(a), a)
a

This works because ROOT converts the Numpy array of doubles into a C-style array of doubles.

But beware! It doesn't _check_ the array type, it _assumes_ the type.

In [ ]:
b = numpy.arange(10, dtype=numpy.int64)    # same number of bits as double, but different meaning
b

In [ ]:
dostuff(len(b), b)
b

It gets even worse if the length is wrong: segmentation faults!

## ROOT and std::vectors

Modern C++ uses `std::vector` to protect both the length and the type, but the data are still contiguous in memory like a Numpy array.

ROOT provides a way to create `std::vectors` in Python and view them as Numpy.

In [ ]:
a = ROOT.std.vector("double")()    # allocate empty
a.push_back(1.1)
a.push_back(2.2)
a.push_back(3.3)
print([x for x in a])

In [ ]:
a_np = numpy.asarray(a)
a_np

This is a _view_ of the _same memory._

In [ ]:
a_np += 100
a_np

In [ ]:
print([x for x in a])

The Numpy array can even be unnamed and short-lived if we use `[:]` to set its data because it is _not_ a copy.

In [ ]:
numpy.asarray(a)[1:] = 999
print([x for x in a])

(See Numpy tutorials for more about `[:]`.)

Using `std::vector`, we can write safer functions.

In [ ]:
tmpname = "dostuffv3"

ROOT.gInterpreter.Declare("""
std::vector<double> %s(std::vector<double> &a, std::vector<double> &b) {
    std::vector<double> out;
    for (int i = 0;  i < a.size() && i < b.size();  i++) {
        out.push_back(a[i] + b[i]);
    }
    return out;
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)

In [ ]:
a = ROOT.std.vector("double")(10)
b = ROOT.std.vector("double")(10)
numpy.asarray(a)[:] = numpy.arange(10) * 1.1
numpy.asarray(b)[:] = 100.0

In [ ]:
c = dostuff(a, b)
numpy.asarray(c)

Unlike Numpy arrays you create yourself, these arrays do not "own" their data. Memory leaks and `new/delete` must be handled entirely on the C++ side.

In [ ]:
numpy.asarray(c).flags

## Other features

<center><img src="images/rvec.png" style="width: 500px;"/></center>
<center><img src="images/rvec1.png"style="width: 500px;"/></center>


**Read a TTree into a NumPy array**
* Branches of arithmetic types (ntuples)


In [ ]:
def make_example():
    root_file = ROOT.TFile("pyroot_example.root", "RECREATE")
    tree = ROOT.TTree("tree", "tutorial")
    x = numpy.empty((1), dtype="float32")
    y = numpy.empty((1), dtype="float32")
    tree.Branch("x", x, "x/F")
    tree.Branch("y", y, "y/F")

    for i in range(4):
        x[0] = i
        y[0] = -i
        tree.Fill()
    root_file.Write()

    return (root_file, x, y), tree

In [ ]:
ROOT.ROOT.EnableImplicitMT()

# Create a ROOT file with a tree and the branches "x" and "y"
_, tree = make_example()

# Read-out full tree as numpy array
array = tree.AsMatrix()
print("Tree converted to a numpy array:\n{}\n".format(array))

In [ ]:
# Get numpy array and according labels of the columns
array, labels = tree.AsMatrix(return_labels=True)
print("Return numpy array and labels:\n{}\n{}\n".format(labels, array))

# Apply numpy methods on the data
print("Mean of the columns retrieved with a numpy method: {}\n".format(
    numpy.mean(array, axis=0)))

# Read only specific branches
array = tree.AsMatrix(columns=["x"])

**Even more powerful way to read TTrees into NumPy**
* All RDataFrame operations available
* Optional parallelism


In [ ]:
df = ROOT.RDataFrame(10) \
         .Define("x", "(int)rdfentry_") \
         .Define("y", "1.f/(1.f+rdfentry_)")

# Next, we want to access the data from Python as Numpy arrays. To do so, the
# content of the dataframe is converted using the AsNumpy method. The returned
# object is a dictionary with the column names as keys and 1D numpy arrays with
# the content as values.
npy = df.AsNumpy()
print("Read-out of the full RDataFrame:\n{}\n".format(npy))

## Interesting and more complex features 


<center><img src="images/cppcall.png" style="width: 500px;"/></center>

<center><img src="images/03-coming-soon-4.png" style="width: 500px;"/></center>

<center><img src="images/03-coming-soon-5.png" style="width: 500px;"/></center>

<center><img src="images/03-coming-soon-6.png" style="width: 500px;"/></center>

## Future features!

<center><img src="images/03-coming-soon-3.png" style="width: 500px;"/></center>

Material used in this tutorial:
* "A more Pythonic, Interoperable and Modern PyROOT" https://indico.cern.ch/event/708041/contributions/3276254/ ACAT 2019
* "PyROOT" https://indico.cern.ch/event/833895/contributions/3577815/ PyHEP 2019